# atmodeller

## Running Tests for test_sulfur.py



In [2]:
from typing import Type

from atmodeller import __version__, debug_logger, debug_file_logger
from atmodeller.interfaces import ConstantConstraint, ConstraintABC
from atmodeller.constraints import (
    FugacityConstraint,
    IronWustiteBufferConstraintHirschmann,
    MassConstraint,
    SystemConstraints,
)
from atmodeller.interior_atmosphere import Species
from atmodeller.interfaces import (
    GasSpecies,
    NoSolubility,
    ThermodynamicData,
    ThermodynamicDataBase,
    ThermodynamicDataJANAF,
)
from atmodeller.interior_atmosphere import InteriorAtmosphereSystem, Planet
from atmodeller.solubilities import (
    BasaltDixonCO2,
    BasaltDixonH2O,
    BasaltH2,
    BasaltS2,
    BasaltS2_Sulfate,
    BasaltS2_Sulfide,
    PeridotiteH2O,
    BasaltLibourelN2
)
from atmodeller.utilities import earth_oceans_to_kg


standard_gibbs_free_energy_of_formation: Type[ThermodynamicDataBase] = ThermodynamicData

kg_to_g = 1000 #g per kg
S_MM: float = 32.065 #g/mol
SO_MM: float = 48.064 #g/mol
S2_MM: float =  64.12 #g/mol
SO2_MM: float = 64.066 #g/mol

debug_file_logger()


### Sulfide and Sulfate Solubility Laws

In [2]:
import numpy as np
def solubility_S2_Sulfide(
       fugacity: float, temperature: float, fugacities_dict: dict[str, float]
    ) -> np.ndarray:
        """Fugacity is fS2."""
        
        logCs: float = 0.225 - (8045.7465 / temperature)
        logS_wtp = logCs - (0.5 * (np.log10(fugacities_dict["O2"]/fugacity)))
        S_wtp = 10**logS_wtp
        ppmw = S_wtp*1.0e4
        #if ppmw >= 1000:
            #msg = "WARNING: SO Sulfide Solubility is getting too high: \n%s" % (ppmw)
            #logger.warning(msg)
            #ppmw = 1000.0
            # raise KeyError(msg)
        return np.array((ppmw, logCs))

In [3]:
def BasaltS2_Sulfate_solubility(
        fugacity: float, temperature: float, fugacities_dict: dict[str, float]
    ) -> np.ndarray:
        """Fugacity is fS2."""
        
        logCs: float = -12.948 + (32333.5635 / temperature)
        logSO4_wtp = logCs + (0.5 * np.log10(fugacity)) + (1.5 * np.log10(fugacities_dict["O2"]))
        SO4_wtp = 10**logSO4_wtp
        S_wtp = SO4_wtp * (32.065/96.06)
        ppmw = (S_wtp)*1e4
        #if ppmw >= 1000:
        #    msg = "WARNING: SO Sulfate Solubility is getting too high: \n%s" % (ppmw)
        #    logger.warning(msg)
        #    ppmw = 1000.0
            # raise KeyError(msg)
        return np.array((ppmw, logCs))

### 1. SO, S2, O2 System, Fugacity Constraints Only

In [ ]:
species: Species = Species(
        [
            GasSpecies(chemical_formula="OS", solubility=NoSolubility()),
            GasSpecies(chemical_formula="S2", solubility=BasaltS2_Sulfide()),
            GasSpecies(chemical_formula="O2", solubility=NoSolubility()),
        ]
    )

In [ ]:
planet: Planet = Planet()
S2_fugacity: float = 1e-5
constraints: SystemConstraints = SystemConstraints([
    FugacityConstraint(species="S2", value=S2_fugacity),
    IronWustiteBufferConstraintHirschmann(),
    ])
system: InteriorAtmosphereSystem = InteriorAtmosphereSystem(species=species, planet=planet)


In [ ]:
system.solve(SystemConstraints(constraints))

In [ ]:
system.fugacities_dict

In [ ]:
system.output

In [ ]:

S_Mass_Total = ((system.output['OS'].mass_in_total * kg_to_g / SO_MM) * S_MM / kg_to_g) + ((system.output['S2'].mass_in_total * kg_to_g / S2_MM) * 2 * S_MM / kg_to_g)
print('Total S mass (atm + melt):', S_Mass_Total)
print('SO Mass (atm + melt):', system.output['OS'].mass_in_total)
print('S2 Mass (atm + melt):', system.output['S2'].mass_in_total)


### 2. SO, S2, SO2, O2 System, Mixed Constraints



#### Sulfide

In [ ]:
species: Species = Species(
        [
            GasSpecies(chemical_formula="SO", solubility=NoSolubility()),
            GasSpecies(chemical_formula="S2", solubility=BasaltS2_Sulfide()),
            GasSpecies(chemical_formula="O2", solubility=NoSolubility()),
            GasSpecies(chemical_formula="SO2", solubility=NoSolubility())
        ]
    )

In [ ]:
surface_temperature: float = 2173 # K

planet: Planet = Planet(surface_temperature=surface_temperature)

mass_S: float = 0.0002 * planet.mantle_mass
print(mass_S)
constraints: SystemConstraints = SystemConstraints([
    MassConstraint(species="S", value=mass_S),
    IronWustiteBufferConstraintHirschmann(log10_shift = 0),
    ])
system: InteriorAtmosphereSystem = InteriorAtmosphereSystem(species=species, planet=planet)


In [ ]:

system.solve(SystemConstraints(constraints))

In [ ]:
system.output

In [ ]:
system.fugacities_dict

In [ ]:
print(system.output['SO'].ppmw_in_melt)
print(system.output['S2'].ppmw_in_melt)
print(system.output['SO2'].ppmw_in_melt)


In [ ]:
S_Mass_Total = ((system.output['SO'].mass_in_total * kg_to_g / SO_MM) * S_MM / kg_to_g) + ((system.output['S2'].mass_in_total * kg_to_g / S2_MM) * 2 * S_MM / kg_to_g) + ((system.output['SO2'].mass_in_total * kg_to_g / SO2_MM) * S_MM / kg_to_g)
print(S_Mass_Total)


In [ ]:
S2_fugacities_dict = {"O2": 1.0269757432683765e-06, "S2": 0.003704873878513222}
print(S2_fugacities_dict)
S2_Sol_test = solubility_S2_Sulfide(0.003704873878513222, 2173, S2_fugacities_dict)
print(S2_Sol_test)

#### Sulfate

In [ ]:
species: Species = Species(
        [
            GasSpecies(chemical_formula="SO", solubility=NoSolubility()),
            GasSpecies(chemical_formula="S2", solubility=BasaltS2_Sulfate()),
            GasSpecies(chemical_formula="O2", solubility=NoSolubility()),
            GasSpecies(chemical_formula="SO2", solubility=NoSolubility())
        ]
    )

In [ ]:
surface_temperature: float = 2173 # K

planet: Planet = Planet(surface_temperature=surface_temperature)

mass_S: float = 0.0002 * planet.mantle_mass
print(mass_S)
SO_fugacity: float = 1.0
constraints: SystemConstraints = SystemConstraints([
    MassConstraint(species="S", value=mass_S),
    #ConstantSystemConstraint(name="fugacity", species="SO", value=SO_fugacity),
    IronWustiteBufferConstraintHirschmann(),
    ])
system: InteriorAtmosphereSystem = InteriorAtmosphereSystem(species=species, planet=planet)

In [ ]:
system.solve(SystemConstraints(constraints))

In [ ]:
system.output

In [ ]:
system.fugacities_dict

In [ ]:
print(system.output['SO'].ppmw_in_melt)
print(system.output['S2'].ppmw_in_melt)
print(system.output['SO2'].ppmw_in_melt)

In [ ]:
S_Mass_Total = ((system.output['SO'].mass_in_total * kg_to_g / SO_MM) * S_MM / kg_to_g) + ((system.output['S2'].mass_in_total * kg_to_g / S2_MM) * 2 * S_MM / kg_to_g) + ((system.output['SO2'].mass_in_total * kg_to_g / SO2_MM) * S_MM / kg_to_g)
print(S_Mass_Total)

print((S_Mass_Total-mass_S)/mass_S)

In [ ]:
S2_fugacities_dict = {"O2": 1.0270757989595792e-06, "S2": 1.525885857348744}
print(S2_fugacities_dict)
S2_Sol_test2 = BasaltS2_Sulfate_solubility(1.525885857348744, 2173, S2_fugacities_dict)
print(S2_Sol_test2)

#### Sulfate and Sulfide (Total S Solubility)

In [ ]:
species: Species = Species(
        [
            GasSpecies(chemical_formula="SO", solubility=NoSolubility()),
            GasSpecies(chemical_formula="S2", solubility=BasaltS2()),
            GasSpecies(chemical_formula="O2", solubility=NoSolubility()),
            GasSpecies(chemical_formula="SO2", solubility=NoSolubility())
        ]
    )

In [ ]:
surface_temperature: float = 2173 # K

planet: Planet = Planet(surface_temperature=surface_temperature)

mass_S: float = 0.0002 * planet.mantle_mass
SO_fugacity = 1e-5
S2_fugacity = 3e-6
print('Mass of Sulfur:', mass_S)
print('Mass Percent of S:', (mass_S/planet.mantle_mass)*100, '%')
print('ppmw of S:', (mass_S/planet.mantle_mass)*100*1E4)
constraints: SystemConstraints = SystemConstraints([
    #ConstantSystemConstraint(name="fugacity", species="SO", value=SO_fugacity),
    #ConstantSystemConstraint(name="fugacity", species="S2", value=S2_fugacity),
    MassConstraint(species="S", value=mass_S),
    IronWustiteBufferConstraintHirschmann(log10_shift=0),
    ])
system: InteriorAtmosphereSystem = InteriorAtmosphereSystem(species=species, planet=planet)

In [ ]:
system.solve(SystemConstraints(constraints))

In [ ]:
system.output

In [ ]:
system.fugacities_dict

In [ ]:

print(system.output['S2'].ppmw_in_melt)



In [ ]:
S_Mass_Total = ((system.output['SO'].mass_in_total * kg_to_g / SO_MM) * S_MM / kg_to_g) + ((system.output['S2'].mass_in_total * kg_to_g / S2_MM) * 2 * S_MM / kg_to_g) + ((system.output['SO2'].mass_in_total * kg_to_g / SO2_MM) * S_MM / kg_to_g)
print(S_Mass_Total)

In [ ]:
S2_fugacities_dict = {"O2": 1.0269757432682946e-06, "S2": 0.0037048732091408743}
TotalS_Sol_test = solubility_S2_Sulfide(0.0037048732091408743, 2173, S2_fugacities_dict)[0] + BasaltS2_Sulfate_solubility(0.0037048732091408743, 2173, S2_fugacities_dict)[0]
print(TotalS_Sol_test)
Sulfide_Sol, Sulfide_logCS = solubility_S2_Sulfide(0.0037048732091408743, 2173, S2_fugacities_dict)
Sulfate_Sol, Sulfate_logCS = BasaltS2_Sulfate_solubility(0.0037048732091408743, 2173, S2_fugacities_dict)
print('Sulfide Solubility:', Sulfide_Sol, 'logC_S2-:', Sulfide_logCS)
print('Sulfate Solubility:', Sulfate_Sol, 'logC_S6+:', Sulfate_logCS)

### Adding C and H species

In [11]:

species: Species = Species(
        [
            #GasSpecies(chemical_formula='H2O', solubility=BasaltDixonH2O()),
            #GasSpecies(chemical_formula='H2',  solubility=BasaltH2()),
            GasSpecies(chemical_formula="SO", solubility=NoSolubility()),
            GasSpecies(chemical_formula="S2", solubility=BasaltS2()),
            GasSpecies(chemical_formula="O2", solubility=NoSolubility()),
            GasSpecies(chemical_formula="SO2", solubility=NoSolubility()),
            GasSpecies(chemical_formula='CO', solubility=NoSolubility()),
            GasSpecies(chemical_formula='CO2', solubility=BasaltDixonCO2())
        ]
    )

species

[21:10:04 - atmodeller.interfaces          - INFO     ] - Creating a GasSpecies: SO (SO)
[21:10:04 - atmodeller.interfaces          - INFO     ] - Creating a GasSpecies: S2 (S2)
[21:10:04 - atmodeller.interfaces          - INFO     ] - Creating a GasSpecies: O2 (O2)
[21:10:04 - atmodeller.interfaces          - INFO     ] - Creating a GasSpecies: SO2 (SO2)
[21:10:04 - atmodeller.interfaces          - INFO     ] - Creating a GasSpecies: CO (CO)
[21:10:04 - atmodeller.interfaces          - INFO     ] - Creating a GasSpecies: CO2 (CO2)


Species([GasSpecies(chemical_formula='SO', name_in_thermodynamic_data='SO', thermodynamic_class=<class 'atmodeller.interfaces.ThermodynamicDataJANAF'>, formula=Formula('SO'), thermodynamic_data=<atmodeller.interfaces.ThermodynamicDataJANAF object at 0x122aa7710>, output=None, solubility=<atmodeller.interfaces.NoSolubility object at 0x122aa4850>, solid_melt_distribution_coefficient=0, fugacity_coefficient=IdealityConstant(name='', species='', value=1.0)),
         GasSpecies(chemical_formula='S2', name_in_thermodynamic_data='S2', thermodynamic_class=<class 'atmodeller.interfaces.ThermodynamicDataJANAF'>, formula=Formula('S2'), thermodynamic_data=<atmodeller.interfaces.ThermodynamicDataJANAF object at 0x11e7a5150>, output=None, solubility=<atmodeller.solubilities.BasaltS2 object at 0x11d5b1e50>, solid_melt_distribution_coefficient=0, fugacity_coefficient=IdealityConstant(name='', species='', value=1.0)),
         GasSpecies(chemical_formula='O2', name_in_thermodynamic_data='O2', thermody

In [12]:
surface_temperature: float = 2173 # K

planet: Planet = Planet(surface_temperature=surface_temperature)

mass_H: float = 0.00108 * planet.mantle_mass * (2/18)
mass_C: float = 0.00014 * planet.mantle_mass 
mass_S: float = 0.0002 * planet.mantle_mass
S2_fugacity: float = 1e-5 
print('Mass of mantle (kg):', planet.mantle_mass)
print('Mass of H:', mass_H)
print('Mass of C:', mass_C)
print('Mass of S:', mass_S)

[21:10:26 - atmodeller.interior_atmosphere - INFO     ] - Creating a new planet
[21:10:26 - atmodeller.interior_atmosphere - INFO     ] - Mantle mass (kg) = 4208261222595110885130240.000000
[21:10:26 - atmodeller.interior_atmosphere - INFO     ] - Mantle melt fraction = 1.000000
[21:10:26 - atmodeller.interior_atmosphere - INFO     ] - Core mass fraction = 0.295335
[21:10:26 - atmodeller.interior_atmosphere - INFO     ] - Planetary radius (m) = 6371000.000000
[21:10:26 - atmodeller.interior_atmosphere - INFO     ] - Planetary mass (kg) = 5972000000000000327155712.000000
[21:10:26 - atmodeller.interior_atmosphere - INFO     ] - Surface temperature (K) = 2173.000000
[21:10:26 - atmodeller.interior_atmosphere - INFO     ] - Surface gravity (m/s^2) = 9.819973
[21:10:26 - atmodeller.interior_atmosphere - INFO     ] - Melt Composition = None


Mass of mantle (kg): 4.208261222595111e+24
Mass of H: 5.0499134671141326e+20
Mass of C: 5.891565711633155e+20
Mass of S: 8.416522445190222e+20


In [13]:

constraints: SystemConstraints = SystemConstraints([
    MassConstraint(species="S", value=mass_S),
    #ConstantSystemConstraint(name="fugacity", species="S2", value=S2_fugacity),
    #MassConstraint(species="H", value=mass_H),
    MassConstraint(species="C", value=mass_C),
    IronWustiteBufferConstraintHirschmann(log10_shift=0)
    ])
system: InteriorAtmosphereSystem = InteriorAtmosphereSystem(species=species, planet=planet)

[21:10:39 - atmodeller.interior_atmosphere - INFO     ] - Creating an interior-atmosphere system
[21:10:39 - atmodeller.interior_atmosphere - INFO     ] - Creating a reaction network
[21:10:39 - atmodeller.interior_atmosphere - INFO     ] - Species = ['SO', 'S2', 'O2', 'SO2', 'CO', 'CO2']
[21:10:39 - atmodeller.interior_atmosphere - INFO     ] - Reactions = 
{0: '0.5 S2 + 1.0 O2 = 1.0 SO2',
 1: '0.5 S2 + 0.5 O2 = 1.0 SO',
 2: '0.5 O2 + 1.0 CO = 1.0 CO2'}


In [18]:
import numpy as np
initial_log10: np.ndarray = np.array([0, 0, -3, -3, -6, -3, 2, 1])
initial_solution: np.ndarray = 10.0**initial_log10

system.solve(SystemConstraints(constraints))

[21:19:02 - atmodeller.interior_atmosphere - INFO     ] - Assembling constraints
[21:19:02 - atmodeller.interior_atmosphere - INFO     ] - Constraints: [MassConstraint(name='mass', species='S', value=8.416522445190222e+20),
 MassConstraint(name='mass', species='C', value=5.891565711633155e+20),
 IronWustiteBufferConstraintHirschmann(name='fugacity',
                                       species='O2',
                                       log10_shift=0)]
[21:19:02 - atmodeller.interior_atmosphere - INFO     ] - 2 additional (mass) constraint(s) are necessary to solve the system
[21:19:02 - atmodeller.interior_atmosphere - INFO     ] - Apply fugacity constraint for O2
[21:19:02 - atmodeller.interior_atmosphere - INFO     ] - Row 00: Reaction 0: 0.5 S2 + 1.0 O2 = 1.0 SO2
[21:19:02 - atmodeller.interior_atmosphere - INFO     ] - Row 01: Reaction 1: 0.5 S2 + 0.5 O2 = 1.0 SO
[21:19:02 - atmodeller.interior_atmosphere - INFO     ] - Row 02: Reaction 2: 0.5 O2 + 1.0 CO = 1.0 CO2
[21:19:02 - 

In [19]:
system.output

{'temperature': 2173,
 'total_pressure_in_atmosphere': 278.12544583726924,
 'mean_molar_mass_in_atmosphere': 0.03074036206312284,
 'SO': GasSpeciesOutput(mass_in_atmosphere=2.469473471074587e+16, mass_in_solid=0.0, mass_in_melt=0.0, moles_in_atmosphere=5.137863969818261e+17, moles_in_melt=0.0, moles_in_solid=0.0, ppmw_in_solid=0.0, ppmw_in_melt=0.0, fugacity=0.003040722714933701, fugacity_coefficient=1.0, pressure_in_atmosphere=0.003040722714933701, volume_mixing_ratio=1.0932918078674552e-05, mass_in_total=2.469473471074587e+16),
 'S2': GasSpeciesOutput(mass_in_atmosphere=4.079752124275351e+16, mass_in_solid=0.0, mass_in_melt=8.41568107287835e+20, moles_in_atmosphere=6.361730190544384e+17, moles_in_melt=1.3122927747683364e+22, moles_in_solid=0.0, ppmw_in_solid=0.0, ppmw_in_melt=199.9800066519789, fugacity=0.0037650388586196962, fugacity_coefficient=1.0, pressure_in_atmosphere=0.0037650388586196962, volume_mixing_ratio=1.3537196667803688e-05, mass_in_total=8.416089048090778e+20),
 'O2':

In [20]:
system.fugacities_dict

{'SO': 0.003040722714933701,
 'S2': 0.0037650388586196962,
 'O2': 1.0437570555403361e-06,
 'SO2': 0.004958587456124795,
 'CO': 230.67647327936177,
 'CO2': 47.437207165120746}

In [21]:
print(system.output['SO'].ppmw_in_melt)
print(system.output['S2'].ppmw_in_melt)
print(system.output['SO2'].ppmw_in_melt)

0.0
199.9800066519789
0.0


In [ ]:
S_Mass_Total = ((system.output['SO'].mass_in_total * kg_to_g / SO_MM) * S_MM / kg_to_g) + ((system.output['S2'].mass_in_total * kg_to_g / S2_MM) * 2 * S_MM / kg_to_g) + ((system.output['SO2'].mass_in_total * kg_to_g / SO2_MM) * S_MM / kg_to_g)
print(S_Mass_Total)

In [ ]:
SO_fugacities_dict = {"O2": 1.04379836369107e-06, "S2": 1e-05}
TotalS_Sol_test = solubility_SO_Sulfide(0.1, 2173, SO_fugacities_dict) + BasaltSO_Sulfate_solubility(0.1, 2173, SO_fugacities_dict)
print(TotalS_Sol_test)

### C, H, N, O and S

In [ ]:

species: Species = Species(
        [
            GasSpecies(chemical_formula='H2O', solubility=BasaltDixonH2O()),
            GasSpecies(chemical_formula='H2',  solubility=BasaltH2()),
            GasSpecies(chemical_formula="SO", solubility=NoSolubility()),
            GasSpecies(chemical_formula="S2", solubility=BasaltS2()),
            GasSpecies(chemical_formula="O2", solubility=NoSolubility()),
            GasSpecies(chemical_formula="SO2", solubility=NoSolubility()),
            GasSpecies(chemical_formula='CO', solubility=NoSolubility()),
            GasSpecies(chemical_formula='CO2', solubility=BasaltDixonCO2()),
            GasSpecies(chemical_formula='N2', solubility=BasaltLibourelN2())
        ]
    )


species

In [ ]:
surface_temperature: float = 2173 # K

planet: Planet = Planet(surface_temperature=surface_temperature)
S2_fugacity: float = 5e-3
mass_H: float = 0.00108 * planet.mantle_mass * (2/18)
mass_C: float = 0.00014 * planet.mantle_mass 
mass_S: float = 0.0002 * planet.mantle_mass
mass_N: float = 0.0000028 * planet.mantle_mass

print('Mass of mantle (kg):', planet.mantle_mass)
print('Mass of H:', mass_H)
print('Mass of C:', mass_C)
print('Mass of S:', mass_S)
print('Mass of N:', mass_N)


In [ ]:

constraints: SystemConstraints = SystemConstraints([
    #MassConstraint(species="S", value=mass_S),
    FugacityConstraint(species="S2", value=S2_fugacity),
    MassConstraint(species="H", value=mass_H),
    MassConstraint(species="C", value=mass_C),
    MassConstraint(species="N", value=mass_N),
    IronWustiteBufferConstraintHirschmann(log10_shift=0)
    ])
system: InteriorAtmosphereSystem = InteriorAtmosphereSystem(species=species, planet=planet)

In [ ]:
system.solve(SystemConstraints(constraints))

In [ ]:
system.output

In [ ]:
system.fugacities_dict

In [ ]:
print(system.output['SO'].ppmw_in_melt)
print(system.output['S2'].ppmw_in_melt)
print(system.output['SO2'].ppmw_in_melt)

In [ ]:
S_Mass_Total = ((system.output['SO'].mass_in_total * kg_to_g / SO_MM) * S_MM / kg_to_g) + ((system.output['S2'].mass_in_total * kg_to_g / S2_MM) * 2 * S_MM / kg_to_g) + ((system.output['SO2'].mass_in_total * kg_to_g / SO2_MM) * S_MM / kg_to_g)
print(S_Mass_Total)

### C, H, N, O, S, Vary Temperature Conditions

In [ ]:
species: Species = Species(
        [
            GasSpecies(chemical_formula='H2O', solubility=BasaltDixonH2O()),
            GasSpecies(chemical_formula='H2',  solubility=BasaltH2()),
            GasSpecies(chemical_formula="SO", solubility=BasaltSO()),
            GasSpecies(chemical_formula="S2", solubility=BasaltS2()),
            GasSpecies(chemical_formula="O2", solubility=NoSolubility()),
            GasSpecies(chemical_formula="SO2", solubility=BasaltSO2()),
            GasSpecies(chemical_formula='CO', solubility=NoSolubility()),
            GasSpecies(chemical_formula='CO2', solubility=BasaltDixonCO2()),
            GasSpecies(chemical_formula='N2', solubility=BasaltLibourelN2())
        ]
    )


species

In [ ]:
surface_temperature: float = 1800 # K

planet: Planet = Planet(surface_temperature=surface_temperature)

mass_H: float = 0.00108 * planet.mantle_mass * (2/18)
mass_C: float = 0.00014 * planet.mantle_mass 
mass_S: float = 0.0002 * planet.mantle_mass
mass_N: float = 0.0000028 * planet.mantle_mass

print('Mass of mantle (kg):', planet.mantle_mass)
print('Mass of H:', mass_H)
print('Mass of C:', mass_C)
print('Mass of S:', mass_S)
print('Mass of N:', mass_N)

In [ ]:

constraints: list[SystemConstraint] = [
    MassConstraint(species="S", value=mass_S),
    MassConstraint(species="H", value=mass_H),
    MassConstraint(species="C", value=mass_C),
    MassConstraint(species="N", value=mass_N),
    IronWustiteBufferConstraintHirschmann(log10_shift=0)
    ]
system: InteriorAtmosphereSystem = InteriorAtmosphereSystem(species=species, planet=planet)

In [ ]:
system.solve(SystemConstraints(constraints))

In [ ]:
system.output

In [ ]:
system.fugacities_dict

In [ ]:
print(system.output['SO'].ppmw_in_melt)
print(system.output['S2'].ppmw_in_melt)
print(system.output['SO2'].ppmw_in_melt)

In [ ]:
SO_fugacities_dict = {"O2": 2.7714826681743157e-09, "S2": 7.286429137857455e-05}
TotalS_Sol_test = solubility_SO_Sulfide(0.1, 1800, SO_fugacities_dict) + BasaltSO_Sulfate_solubility(0.1, 1800, SO_fugacities_dict)
print(TotalS_Sol_test)

### C, H, N, O, S, Vary Constraints

In [ ]:
species: Species = Species(
        [
            GasSpecies(chemical_formula='H2O', solubility=BasaltDixonH2O()),
            GasSpecies(chemical_formula='H2',  solubility=BasaltH2()),
            GasSpecies(chemical_formula="SO", solubility=BasaltSO()),
            GasSpecies(chemical_formula="S2", solubility=BasaltS2()),
            GasSpecies(chemical_formula="O2", solubility=NoSolubility()),
            GasSpecies(chemical_formula="SO2", solubility=BasaltSO2()),
            GasSpecies(chemical_formula='CO', solubility=NoSolubility()),
            GasSpecies(chemical_formula='CO2', solubility=BasaltDixonCO2()),
            GasSpecies(chemical_formula='N2', solubility=BasaltLibourelN2())
        ]
    )


species

In [ ]:
surface_temperature: float = 2173 # K

planet: Planet = Planet(surface_temperature=surface_temperature)

mass_H: float = 0.00108 * planet.mantle_mass * (2/18)
mass_C: float = 0.00014 * planet.mantle_mass 
mass_S: float = 0.0002 * planet.mantle_mass
mass_N: float = 0.0000028 * planet.mantle_mass

print('Mass of mantle (kg):', planet.mantle_mass)
print('Mass of H:', mass_H)
print('Mass of C:', mass_C)
print('Mass of S:', mass_S)
print('Mass of N:', mass_N)

In [ ]:

constraints: list[SystemConstraint] = [
    ConstantSystemConstraint(name="fugacity", species="S2", value=5e-3),
    MassConstraint(species="H", value=mass_H),
    MassConstraint(species="C", value=mass_C),
    MassConstraint(species="N", value=mass_N),
    IronWustiteBufferConstraintHirschmann(log10_shift=0)
    ]
system: InteriorAtmosphereSystem = InteriorAtmosphereSystem(species=species, planet=planet)

In [ ]:
system.solve(SystemConstraints(constraints))

In [ ]:
system.output

In [ ]:
system.fugacities_dict

In [ ]:
print(system.output['SO'].ppmw_in_melt)
print(system.output['S2'].ppmw_in_melt)
print(system.output['SO2'].ppmw_in_melt)

In [ ]:
SO_fugacities_dict = {"O2": 1.0439298480079078e-06, "S2": 5e-3}
TotalS_Sol_test = solubility_SO_Sulfide(0.1, 2173, SO_fugacities_dict) + BasaltSO_Sulfate_solubility(0.1, 2173, SO_fugacities_dict)
print(TotalS_Sol_test)